In [ ]:
import numpy as np
import pandas as pd
import fastf1
from fastf1 import utils
from collections import defaultdict, Counter
import statsmodels.api as sm
from src.utils import get_acc_df, fuel_correction, compare_car_speeds, compute_track_dominance_multi
from src.plotset import setup_plot, save_fig, plot_track_dominance

from fastf1 import plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.collections import LineCollection
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,16,'Q')
session.load()

In [ ]:
corners = session.get_circuit_info().corners
rotation = session.get_circuit_info().rotation

In [ ]:
ref_lap = session.laps.pick_drivers('VER').pick_fastest().get_telemetry().copy()
comp_lap = session.laps.pick_drivers('NOR').pick_fastest().get_telemetry().copy()

In [ ]:
mult = ref_lap.Distance.iloc[-1]/comp_lap.Distance.iloc[-1]

In [ ]:
ref_dist = ref_lap.Distance.to_numpy()
ref_time = ref_lap.Time.dt.total_seconds().to_numpy()
comp_dist = (comp_lap.Distance * mult).to_numpy()
comp_time = comp_lap.Time.dt.total_seconds().to_numpy()

In [ ]:
comp_time = np.interp(ref_dist, comp_dist, comp_time)

In [ ]:
delta = comp_time - ref_time

In [ ]:
dominance = pd.Series(delta).diff().rolling(window=5,center=True).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(ref_lap['Distance'],ref_lap['Speed'],color="#0400FF",linewidth=2)
ax.plot(comp_lap['Distance'],comp_lap['Speed'],color="#FF8000",linewidth=2)

v_min = ref_lap['Speed'].min()
v_max = ref_lap['Speed'].max()
ax.vlines(x=corners['Distance'], ymin=v_min-50, ymax=v_max+30,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax.text(corner['Distance'], v_max+30, txt,
            va='center_baseline', ha='center', size=15)


ax.set_xlim(ref_dist[0],ref_dist[-1])
ax.set_ylim(ref_lap['Speed'].min() - 50, ref_lap['Speed'].max() + 50)
ax.grid(visible=False)

ax2 = ax.twinx()
ax2.plot(ref_lap['Distance'],delta,ls='--',color='w')
ax2.set_ylim(-0,1)
ax2.grid(visible=False)
ax2.axhline(y=0,lw=1,ls='--',color='#444444')

In [ ]:
from scipy.signal import savgol_filter

def long_acc(lap, window=6, polyorder=2):

    dv = (lap.Speed * (5/18)).diff()
    dt = lap.Time.dt.total_seconds().diff()
    a = dv / dt
    a_g = a / 9.81

    # Smooth with Savitzky–Golay filter
    return savgol_filter(a_g.fillna(0), window_length=window, polyorder=polyorder)

In [ ]:
comp_lap['Distance'] = comp_dist

In [ ]:
ref_lap['LA'] = long_acc(ref_lap,window=11,polyorder=2)
comp_lap['LA'] = long_acc(comp_lap,window=11,polyorder=2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(comp_lap['Distance'],comp_lap['LA'],color="#FF8000",linewidth=3)
ax.plot(ref_lap['Distance'],ref_lap['LA'],color="#0400FF",linewidth=3)


v_min = ref_lap['LA'].min()
v_max = ref_lap['LA'].max()
ax.vlines(x=corners['Distance'], ymin=v_min-1, ymax=v_max+1,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax.text(corner['Distance'], v_max+1, txt,
            va='center_baseline', ha='center', size=15)


ax.set_xlim(ref_dist[0],ref_dist[-1])
# ax.set_ylim(-6, 2)

In [ ]:
import numpy as np
from scipy.signal import savgol_filter

def compute_lateral_acc(df, window_length=21, polyorder=2, outlier_threshold=100, xy_window=21, xy_poly=2):

    df = df.copy()

    # --- Pre-filter X, Y, Distance ---
    n = len(df)
    w = min(xy_window, n if n % 2 == 1 else n-1)  # ensure odd, <= n
    if w < 3:
        w = 3

    x = savgol_filter(df['X'].to_numpy(), w, xy_poly)
    y = savgol_filter(df['Y'].to_numpy(), w, xy_poly)
    s = savgol_filter(df['Distance'].to_numpy(), w, xy_poly)

    # Step 1. Heading angle
    delta_x = np.diff(x, prepend=x[0])
    delta_y = np.diff(y, prepend=y[0])
    heading = np.arctan2(delta_y, delta_x)

    # Step 2. Heading change and distance delta
    d_heading = np.diff(heading, prepend=heading[0])
    ds = np.diff(s, prepend=s[0])

    # Step 3. Radius of curvature
    radius = np.divide(ds, d_heading, 
                       out=np.full_like(ds, np.nan, dtype=float), 
                       where=d_heading != 0)

    # Step 4. Lateral acceleration (v²/R)
    v = df['Speed'].to_numpy() / 3.6  # km/h → m/s
    lat_acc = np.divide(v**2, radius, 
                        out=np.full_like(v, np.nan, dtype=float),
                        where=~np.isnan(radius))

    # Step 5. Remove unrealistic values
    lat_acc[np.abs(lat_acc) > outlier_threshold] = np.nan

    # Step 6. Smooth LatAcc and convert to g
    lat_acc_filt = savgol_filter(np.nan_to_num(lat_acc), 
                                 window_length=window_length, 
                                 polyorder=polyorder)

    return lat_acc_filt / 9.81  # in g

In [ ]:
comp_lap['Distance'] = comp_dist

In [ ]:
ref_lap['lat_acc'] = compute_lateral_acc(ref_lap)
comp_lap['lat_acc'] = compute_lateral_acc(comp_lap)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(ref_lap['Distance'],ref_lap['lat_acc'],color="#FF8000",linewidth=3)
ax.plot(comp_lap['Distance'],comp_lap['lat_acc'],color="#0400FF",linewidth=3)

v_min = ref_lap['lat_acc'].min()
v_max = ref_lap['lat_acc'].max()
ax.vlines(x=corners['Distance'], ymin=v_min-2, ymax=v_max+2,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax.text(corner['Distance'], v_max+1.6, txt,
            va='center_baseline', ha='center', size=12)

ax.set_xlim(ref_dist[0],ref_dist[-1])
ax.set_ylim(-5,5)
ax.grid(visible=False)